Clean data

In [249]:
import pandas as pd

In [279]:
# Get limitations
with open(r"Cleaned_datasets\limitations.txt", "r") as f:
    limitations = f.readlines()[-1].split(',')
    start_date, end_date = [pd.to_datetime(date) for date in limitations]

Clean energy consumption dataset

In [251]:
energy_df = pd.read_csv(r"C:\Users\wong2\PycharmProjects\Data_sci_pw\Datasets\Uncleaned_datasets\energy_consumption.csv")

In [252]:
# Rename column
energy_df = energy_df.rename(columns={'ds': 'DateTime'})

In [253]:
# Convert ds column to datetime format
energy_df['DateTime'] = pd.to_datetime(energy_df['DateTime'])

In [254]:
# Only get dates within limitations
energy_df = energy_df[(energy_df['DateTime'] >= start_date) & (energy_df['DateTime'] <= end_date)]

In [255]:
# Write to csv
energy_df.to_csv(r"C:\Users\wong2\PycharmProjects\Data_sci_pw\Datasets\Cleaned_datasets\energy_consumption.csv", index = False)

Clean holidays dataset

In [256]:
# Load dataset
holiday_df = pd.read_csv(r"C:\Users\wong2\PycharmProjects\Data_sci_pw\Datasets\Uncleaned_datasets\holiday.csv")

In [257]:
# Rename Date col to DateTime
holiday_df = holiday_df.rename(columns={"Date": "DateTime"})

In [258]:
# Change format to datetime
holiday_df['DateTime'] = pd.to_datetime(holiday_df["DateTime"])

In [259]:
# Only get dates within limitations
holiday_df = holiday_df[(holiday_df['DateTime'] >= start_date) & (holiday_df['DateTime'] <= end_date)]

In [260]:
# Make each day a num
day_map = {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7}

# add a new column to the dataframe and map the days of the week to numerical labels
holiday_df['Day'] = holiday_df['Day'].map(day_map)

In [261]:
# Clean the day and holidays to a number
# Get unique holidays
unique_holidays = holiday_df['Holiday'].unique()

# Create a dictionary of unique numbers for each holiday
holiday_dict = {}
for i, holiday in enumerate(unique_holidays):
    holiday_dict[holiday] = i+1

# Map the dictionary to the holiday column of the DataFrame
holiday_df['Holiday'] = holiday_df['Holiday'].map(holiday_dict)

In [262]:
# Write to csv
holiday_df.to_csv(r"C:\Users\wong2\PycharmProjects\Data_sci_pw\Datasets\Cleaned_datasets\holiday.csv", index = False)

Merge amk, tuas, and changi datasets

In [282]:
# load dfs
df_locationA = pd.read_csv(r"C:\Users\wong2\PycharmProjects\Data_sci_pw\Datasets\Uncleaned_datasets\angmokio.csv")

df_locationB= pd.read_csv(r"C:\Users\wong2\PycharmProjects\Data_sci_pw\Datasets\Uncleaned_datasets\changi.csv")

df_locationC= pd.read_csv(r"C:\Users\wong2\PycharmProjects\Data_sci_pw\Datasets\Uncleaned_datasets\tuassouth.csv")

In [283]:
# Merge dfs
merged_df = pd.concat([df_locationA, df_locationB, df_locationC])

In [284]:
# Ground the dfs and mean the vals like temp
weather_df = merged_df.groupby(['Year', 'Month', 'Day']).mean(numeric_only=True).reset_index()

In [285]:
# Convert weather datetime df into pd datetime
weather_df['DateTime'] = pd.to_datetime(weather_df[['Year', 'Month', 'Day']].astype(str).apply('-'.join, 1) + ' ' + '00:00:00')


weather_df = weather_df.drop(['Year', 'Month', 'Day'], axis=1)

In [286]:
# Make the Datetime col the first col
weather_df.insert(0, 'DateTime', weather_df.pop('DateTime'))

In [290]:
# Drop unnamed col
weather_df = weather_df.drop("Unnamed: 0", axis=1)

In [291]:
# Remove dates out of allowed range
weather_df = weather_df[(weather_df['DateTime'] >= pd.to_datetime(start_date)) & (weather_df['DateTime'] <= pd.to_datetime(end_date))]


In [292]:
# Write to csv
weather_df.to_csv(r"C:\Users\wong2\PycharmProjects\Data_sci_pw\Datasets\Cleaned_datasets\weather.csv", index=False)

Parse code into final dataset

In [293]:
# Load weather df
weather_df = pd.read_csv(r"C:\Users\wong2\PycharmProjects\Data_sci_pw\Datasets\Cleaned_datasets\weather.csv")

In [294]:
# Load in holiday file
holiday_df = pd.read_csv(r"C:\Users\wong2\PycharmProjects\Data_sci_pw\Datasets\Cleaned_datasets\holiday.csv")

In [295]:
# Load energy consumption file
energy_df = pd.read_csv(r"C:\Users\wong2\PycharmProjects\Data_sci_pw\Datasets\Cleaned_datasets\energy_consumption.csv")

In [296]:
# Merge the dfs
merged_df_1 = pd.merge(holiday_df, weather_df, on='DateTime', how='left')
merged_df_2 = pd.merge(holiday_df, energy_df, on='DateTime', how='left')


In [297]:
# Concatenate the two merged dataframes
final_df = pd.concat([merged_df_1, merged_df_2], axis=0)

In [298]:
# Write to CSV file
final_df.to_csv(r"C:\Users\wong2\PycharmProjects\Data_sci_pw\Datasets\final_dataset.csv", index=False)

In [299]:
final_df

,DateTime,Day,Holiday,Daily Rainfall Total (mm),Highest 30 min Rainfall (mm),Highest 60 min Rainfall (mm),Highest 120 min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h),y
0,2017-01-01,7,1,4.333333,2.933333,3.266667,3.533333,27.066667,31.200000,25.233333,7.333333,32.966667,NaN
1,2017-01-02,1,1,15.800000,7.733333,10.733333,13.200000,26.333333,29.766667,24.633333,4.566667,22.933333,NaN
2,2017-01-28,6,2,1.333333,1.000000,1.133333,1.266667,26.266667,29.600000,24.600000,7.566667,33.833333,NaN
3,2017-01-29,7,2,0.200000,0.066667,0.133333,0.200000,26.466667,28.566667,24.700000,6.400000,22.300000,NaN
4,2017-04-14,5,3,27.266667,21.933333,26.066667,27.200000,27.233333,31.400000,24.233333,5.266667,34.833333,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,2019-05-01,3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,2019-04-19,5,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,2019-02-06,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,2019-02-05,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
